In [123]:
import numpy as np
import six
import albumentations as A
import numpy as np
import cv2
from skimage.transform import AffineTransform, warp
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
from loader import LoadDataset
from transf import * 
from torch.utils.data.dataloader import DataLoader
import torch
# already included into loader

import sys
sys.path.append('/home/jbonato/Documents/cvpr_clvision_challenge/')
from core50.dataset import CORE50
dataset = CORE50(root='/home/jbonato/Documents/cvpr_clvision_challenge/core50/data/', scenario='ni',preload=True)


Loading data...
Loading paths...
Loading LUP...
Loading labels...
preparing CL benchmark...


In [132]:
_,labels,_ = dataset.next()
a,b = np.unique(labels,return_counts=True)
plt.scatter(a,b)

Batch number: 8


StopIteration: 

In [6]:
import numpy
import six
import torch
from torch.utils.data.dataset import Dataset
#import albumentations as A
import numpy as np
#import cv2
#from skimage.transform import AffineTransform, warp
import numpy as np
#import pandas as pd
import gc

class DatasetMixin(Dataset):

    def __init__(self, transform=None):
        self.transform = transform
        print('il cristo')

    def __getitem__(self, index):
        """Returns an example or a sequence of examples."""
        if torch.is_tensor(index):
            index = index.tolist()
        if isinstance(index, slice):
            current, stop, step = index.indices(len(self))
            return [self.get_example_wrapper(i) for i in
                    six.moves.range(current, stop, step)]
        elif isinstance(index, list) or isinstance(index, numpy.ndarray):
            return [self.get_example_wrapper(i) for i in index]
        else:
            return self.get_example_wrapper(index)

    def __len__(self):
        """Returns the number of data points."""
        raise NotImplementedError

    def get_example_wrapper(self, i):
        """Wrapper of `get_example`, to apply `transform` if necessary"""
        example = self.get_example(i)
        
        if self.transform:
            example = self.transform(example)
   
        return example

    def get_example(self, i):
        """Returns the i-th example.

        Implementations should override it. It should raise :class:`IndexError`
        if the index is invalid.

        Args:
            i (int): The index of the example.

        Returns:
            The i-th example.

        """
        raise NotImplementedError

class LoadDataset(DatasetMixin):
    def __init__(self, images, labels=None, transform=None, indices=None):
        super(LoadDataset, self).__init__(transform=transform)
        self.images = images
        self.labels = labels
        if indices is None:
            indices = np.arange(len(images))
        self.indices = indices
        self.train = labels is not None

    def __len__(self):
        """return length of this dataset"""
        return len(self.indices)

    def get_example(self, i):
        """Return i-th data"""
        i = self.indices[i]
        x = self.images[i]
        
        # scale to [0,1] interval
        x = x/255
        
        # normalize
        x[ :, :, 0] = ((x[ :, :, 0] - 0.485) / 0.229)
        x[ :, :, 1] = ((x[ :, :, 1] - 0.456) / 0.224)
        x[ :, :, 2] = ((x[ :, :, 2] - 0.406) / 0.225)
        
        # Swap channel dimension to fit the caffe format (c, w, h)
        x = np.transpose(x, (2, 0, 1))        
        if self.train:
            y = self.labels[i]
            return x, y
        else:
            return x

In [57]:
q =np.arange(20).reshape(4,5)
np.flip(q,axis=0)

array([[15, 16, 17, 18, 19],
       [10, 11, 12, 13, 14],
       [ 5,  6,  7,  8,  9],
       [ 0,  1,  2,  3,  4]])

In [7]:
tr =Transform(affine=True, train=True,blur_ratio=0.5, cutout_ratio=0.5, ssr_ratio=0.5)
n_dataset= data.shape[0]
debug = False
train_data_size = 800 if debug else int(n_dataset * 0.8)
test_data_size = 10 if debug else int(n_dataset * 0.1)
valid_data_size = 100 if debug else int(n_dataset - train_data_size-test_data_size)


perm = np.random.RandomState(777).permutation(n_dataset)

    
train_set = LoadDataset(data,labels,transform=tr,indices=perm[:train_data_size])
val_set = LoadDataset(data,labels,indices=perm[train_data_size:train_data_size+valid_data_size])
test_set = LoadDataset(data,labels,indices=perm[train_data_size+valid_data_size:train_data_size+valid_data_size+test_data_size])

batch_size=32#64
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

wwwwwwwwwww
il cristo
il cristo
il cristo


In [9]:
device = torch.device('cuda:0')
for inputs, labels in train_loader:
    print(inputs.size())

(3, 128, 128) 45.0
(3, 128, 128) 4.0
(3, 128, 128) 39.0
(3, 128, 128) 38.0
(3, 128, 128) 14.0
(3, 128, 128) 36.0
(3, 128, 128) 30.0
(3, 128, 128) 36.0
(3, 128, 128) 18.0
(3, 128, 128) 29.0
(3, 128, 128) 37.0
(3, 128, 128) 46.0
(3, 128, 128) 5.0
(3, 128, 128) 30.0
(3, 128, 128) 20.0
(3, 128, 128) 43.0
(3, 128, 128) 8.0
(3, 128, 128) 25.0
(3, 128, 128) 46.0
(3, 128, 128) 41.0
(3, 128, 128) 38.0
(3, 128, 128) 6.0
(3, 128, 128) 5.0
(3, 128, 128) 35.0
(3, 128, 128) 23.0
(3, 128, 128) 19.0
(3, 128, 128) 29.0
(3, 128, 128) 15.0
(3, 128, 128) 27.0
(3, 128, 128) 28.0
(3, 128, 128) 15.0
(3, 128, 128) 22.0
torch.Size([32, 3, 128, 128])
(3, 128, 128) 37.0
(3, 128, 128) 33.0
(3, 128, 128) 31.0
(3, 128, 128) 29.0
(3, 128, 128) 25.0
(3, 128, 128) 10.0
(3, 128, 128) 40.0
(3, 128, 128) 42.0
(3, 128, 128) 26.0
(3, 128, 128) 49.0
(3, 128, 128) 33.0
(3, 128, 128) 26.0
(3, 128, 128) 8.0
(3, 128, 128) 26.0
(3, 128, 128) 26.0
(3, 128, 128) 33.0
(3, 128, 128) 19.0
(3, 128, 128) 26.0
(3, 128, 128) 3.0
(3, 128,

KeyboardInterrupt: 